In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow #לחילוץ פריים
from IPython.display import HTML #לצפייה בסרטון
from base64 import b64encode #לצפייה בסרטון

In [ ]:
# העלו לקולב את הסרטון של הבלון
# צרו משתנה בשם cap
#בעזרת cv2.VideoCapture
#שיכיל את הסרט


In [ ]:
#מה הטיפוס של cap?

In [ ]:
#צפייה בסרטון
path = ## הוסיפו כאן את הנתיב לסרטון שלכם
mp4 = open(path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
  <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
# בעזרת המתודה read
#חלצו frame ראשון
#שימו לב שמתקבלים מהמתודה שני משתנים (ret, frame)ולא אחד
#הקפידו לקרוא לפריים שלכם frame


In [ ]:
# צרו משתנה בשם hsv_frame
#שיהיה תוצאת ההמרה של הפריים  שלכם לפורמט HSV
#בעזרת הפונקציה cv2.cvtColor()
#שמקבלת את הפריים ואת הפורמט: frame, cv2.COLOR_BGR2HSV


In [ ]:
 #הריצו את הקוד הבא להצגת התמונה המומרת
 #הוסיפו גם כותרת, גריד מידי 50 םיקסלים ואת הצירים עצמם כדי שתוכלו לזהות את מיקום הבלון
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))


In [ ]:
# תוך קורדינטות עם התמונה על פי ערכים
# הגדרת הקואורדינטות שנבחרו (למשל)
x, y = # הוסיפו את הקורדינטות שבחרתם

# הגבלת גבולות החיתוך כדי לא לצאת מהתמונה
h, w = hsv.shape[:2]
crop_size = 25  # כרגע הפילטר מוגדר עם המספר 25 לגודל של 50 על 50. יתכן שתצטרכו לשחק עם זה

x_start = max(0, x - crop_size)
x_end = min(w, x + crop_size)
y_start = max(0, y - crop_size)
y_end = min(h, y + crop_size)

# חיתוך האזור מתוך תמונת HSV (שהומרה קודם)
crop = hsv[y_start:y_end, x_start:x_end]

In [ ]:
# הצגת מה שחתכם
#בדקו שאתם מקבלים בתוצאה רק "כתום"
# המרה מ-HSV ל-RGB לצורך הצגה נכונה
crop_rgb = cv2.cvtColor(crop, cv2.COLOR_HSV2RGB)
# הצגה
plt.imshow(crop_rgb)
plt.axis("off")

In [ ]:
pixels = crop.reshape(-1, 3)  # הפיכת המטריצה לרשימת פיקסלים
# חישוב ממוצע, מינימום ומקסימום לכל ערוץ (H, S, V)
mean_hsv = np.mean(pixels, axis=0)
min_hsv = np.min(pixels, axis=0)
max_hsv = np.max(pixels, axis=0)

print("ממוצע HSV:", mean_hsv)
print("מינימום HSV:", min_hsv)
print("מקסימום HSV:", max_hsv)

In [ ]:
# קביעת טווח מעט רחב יותר ממה שהתקבל במינ מקס משיקולי אור\צל וכו'
#שימו לב- יתכן שתרצו לשחק קצת עם הערכים
#ו\או עם משהו אחר פה, תלוי בתוצאות

lower = np.array([
    max(0, min_hsv[0] - 2),     # Hue תחתון - טיפה פחות
    max(0, min_hsv[1] - 20),    # Saturation - גמיש יותר
    max(0, min_hsv[2] - 20)     # Value - גם כאן
])

upper = np.array([
    min(179, (max_hsv[0] + 2),   # Hue עליון - טיפה יותר
    min(255, max_hsv[1] + 30),  # Saturation
    min(255, max_hsv[2] + 30)   # Value
])


In [ ]:
#ודאו את upper & lower שקיבלתם

In [ ]:
#צרו משתנה בשם mask
#בעזרת הפונקציה cv2.inRange
#המקבלת את הפריים בפורמט hvs ואת הגבול העליון והתחתון


In [ ]:
#בעזרת cv2.bitwise_and
#שמקבל פעמיים את הפריים  ופעם אחת את mask
#צרו משתנה בשם rresult
result = cv2.bitwise_and(frame, frame, mask=mask)

In [ ]:
#תצוגת המסיכה:
plt.imshow(mask, cmap="gray")
plt.title("the mask (inRange)")
plt.axis("off")

In [ ]:
#:#תצוגת הבלון בלבד
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

plt.imshow(result_rgb)
plt.title("Just the object")
plt.axis("off")

In [ ]:
#הריצו את הקוד הבא להצגת הקונטור
# מציאת קונטורים
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# בחירת הקונטור הגדול ביותר
largest_contour = max(contours, key=cv2.contourArea)

# ציור הקונטורעל התמונה
frame_with_contour = frame.copy()
cv2.drawContours(frame_with_contour, [largest_contour], -1, (0, 255, 0), 2)  # ירוק

# המרה ל-RGB לצורך הצגה עם matplotlib
frame_rgb = cv2.cvtColor(frame_with_contour, cv2.COLOR_BGR2RGB)
#  הצגה
plt.imshow(frame_rgb)
plt.axis("off")
plt.show()


In [ ]:
# צור משתנה בשם M
#שהוא מרכז הקונטור
#שהו א תוצאת הפונקציה cv2.moments
#לאחר שקיבלה את הקונטור הגדול ביותר


In [ ]:
#חישוב המרכז
center_x = #הוסף חישוב כאן
center_y = #הוסף חישוב כאן
center = (center_x, center_y)


In [ ]:
# הרץ את הקוד הבא כדי לראות את מרכז הקנטור
# ציור עיגול במרכז
frame_with_center = frame.copy()
cv2.circle(frame_with_center, center, 5, (0, 0, 255), -1)  # עיגול אדום

# ציור גם של הקונטור מסביב לכדור (כמו קודם)
cv2.drawContours(frame_with_center, [largest_contour], -1, (0, 255, 0), 2)  # ירוק

# הצגה
plt.imshow(cv2.cvtColor(frame_with_center, cv2.COLOR_BGR2RGB))
plt.title("Center(Red) and contour (Green)")
plt.show()

In [ ]:
#הרץ את הקוד הבא ליצירת
# יצירת Kalman Filter בגודל 4x2:
# מצב = [x, y, dx, dy] ← מיקום ומהירות
# מדידה = [x, y]
kalman = cv2.KalmanFilter(4, 2)

# מטריצת המעבר (transition) ← איך המצב מתפתח עם הזמן
kalman.transitionMatrix = np.array([
    [1, 0, 1, 0],  # x ← x + dx
    [0, 1, 0, 1],  # y ← y + dy
    [0, 0, 1, 0],  # dx ← dx
    [0, 0, 0, 1]   # dy ← dy
], dtype=np.float32)

# מטריצת מדידה ← איך המצב משפיע על המדידה בפועל
kalman.measurementMatrix = np.array([
    [1, 0, 0, 0],  # מודדים רק x
    [0, 1, 0, 0]   # מודדים רק y
], dtype=np.float32)

# רעש מדידה ורעש תנועה
#קלמן סומך פחות על התחזית ככל שהערך גבוה. מטריצה  3 על 3 עם 1 באלכסון רעש בתנועה
kalman.processNoiseCov = np.eye(4, dtype=np.float32) * 0.03
# ככל שהערך גבוה קלמן סמך פחות על הזיהוי רעש במדידה
kalman.measurementNoiseCov = np.eye(2, dtype=np.float32) * 1

# נגדיר את המצב ההתחלתי כמרכז הקונטור שמצאנו
kalman.statePre = np.array([[np.float32(center[0])],
                            [np.float32(center[1])],
                            [0], [0]])


In [ ]:
# הרצת הקוד הבא תאפשר צפיה בפריימים הראשונים- תחזית לעומת מציאות. שימו לב מה איכות התחזית בשלב זה
initialized = False
frames = []

cap.release()
cap = cv2.VideoCapture("baloon2.mp4")

# נקרא 30 פריימים ראשונים לדוגמה
for _ in range(30):
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest)

        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            center = np.array([[np.float32(cx)], [np.float32(cy)]])

            if not initialized:
                kalman.statePre = np.array([[cx], [cy], [0], [0]], dtype=np.float32)
                initialized = True

            prediction = kalman.predict()
            pred_x = int(prediction[0][0])
            pred_y = int(prediction[1][0])

            kalman.correct(center)

            # ציור
            cv2.drawContours(frame, [largest], -1, (0, 255, 0), 2)
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)  # אדום – מיקום אמיתי
            cv2.circle(frame, (pred_x, pred_y), 10, (255, 0, 0), -1)  # כחול – תחזית

    # שמירה להצגה
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

cap.release()

# הצגה של 6 פריימים לדוגמה
plt.figure(figsize=(15, 10))
for i in range(6):
    plt.subplot(2, 3, i + 1)
    plt.imshow(frames[i])
    plt.title(f"Frame {i + 1}")
    plt.axis("off")
plt.tight_layout()
plt.show()


In [ ]:
# הרצת הקוד הזה תראה את התחזית מידי 25 פריימים לאורך כל הסרטון- האם נצפה שיפור?
cap.release()
cap = cv2.VideoCapture("baloon2.mp4")

initialized = False
frame_index = 0
display_index = 1

while True:
    ret, frame = cap.read()
    if not ret:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest)

        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            center = np.array([[np.float32(cx)], [np.float32(cy)]])

            if not initialized:
                kalman.statePre = np.array([[cx], [cy], [0], [0]], dtype=np.float32)
                initialized = True

            prediction = kalman.predict()
            kalman.correct(center)

            # ציור תמיד – גם אם לא נשמר
            cv2.drawContours(frame, [largest], -1, (0, 255, 0), 2)  # ירוק
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)         # אדום
            cv2.circle(frame, (int(prediction[0][0]), int(prediction[1][0])), 10, (255, 0, 0), 2)  # כחול

    # רק כל 50 פריימים – נשמור ונציג
    if frame_index % 25 == 0:
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(6, 4))
        plt.imshow(rgb)
        plt.title(f"Frame {frame_index}")
        plt.axis("off")
        plt.show()

    frame_index += 1

cap.release()


In [ ]:
#מטיבי לכת?
#נשאר זמן?
#תעשו תחזית עם רגרסיה לינארי כדי להשוות